In [1]:
import Pkg; 

if split(pwd(),"/")[end] == "notebooks"
    cd(joinpath(@__DIR__, "../"))
    Pkg.activate("Project.toml")
end

using MorphoMol

using CairoMakie
using LinearAlgebra
using Random
using Rotations
using StaticArrays
using Distributions
using Distances
using BenchmarkTools
using JLD2

  Activating project at `~/Desktop/Doktor/MorphoMol/MorphoMolMonteCarlo`


In [2]:
function get_flat_realization(x, template_mol)
    n_mol = length(x) ÷ 6
    [(hvcat((n_mol), [exp(Rotations.RotationVecGenerator(x[i:i+2]...)) * template_mol .+ x[i+3:i+5] for i in 1:6:length(x)]...)...)...]
end

get_flat_realization (generic function with 1 method)

In [3]:
function rotation_and_translation_gradient!(∇E, x, ∇FSol, template_mol)
    n_atoms_per_mol = size(template_mol)[2]
    n_mol = length(x) ÷ 6
    for i in 1:n_mol        
        R = exp(Rotations.RotationVecGenerator(x[(i-1)*6 + 1:(i-1)*6 + 3]...))
        ∇E[(i-1) * 6 + 1] = 0.5 * sum([-v[2]*(R[3,:] ⋅ w) + v[3]*(R[2,:] ⋅ w) for (v,w) in [(∇FSol[:,:,i][:,j], template_mol[:,j]) for j in 1:n_atoms_per_mol]])
        ∇E[(i-1) * 6 + 2] = 0.5 * sum([v[1]*(R[3,:] ⋅ w) - v[3]*(R[1,:] ⋅ w) for (v,w) in [(∇FSol[:,:,i][:,j], template_mol[:,j]) for j in 1:n_atoms_per_mol]])
        ∇E[(i-1) * 6 + 3] = 0.5 * sum([-v[1]*(R[2,:] ⋅ w) + v[2]*(R[1,:] ⋅ w) for (v,w) in [(∇FSol[:,:,i][:,j], template_mol[:,j]) for j in 1:n_atoms_per_mol]])
        ∇E[(i-1) * 6 + 4:(i-1) * 6 + 6] = sum([∇FSol[:,j,i] for j in 1:n_atoms_per_mol])
    end
    ∇E
end

function solvation_free_energy_gradient!(∇E, x, template_mol, radii, rs, pf, overlap_slope)
    n_atoms_per_mol = size(template_mol)[2]
    n_mol = length(x) ÷ 6
    flat_realization = get_flat_realization(x, template_mol)
    _, dvol, dsurf, dmean, dgauss, dlol = MorphoMol.Energies.get_geometric_measures_and_overlap_value_with_derivatives(
        flat_realization,
        n_atoms_per_mol,
        radii,
        rs,
        0.0,
        overlap_slope
    )
    ∇FSol = reshape(pf[1] * dvol + pf[2] * dsurf + pf[3] * dmean + pf[4] * dgauss + dlol, (3, n_atoms_per_mol, n_mol))
    rotation_and_translation_gradient!(∇E, x, ∇FSol, template_mol)
end


solvation_free_energy_gradient! (generic function with 1 method)

In [4]:
function solvation_free_energy(x::Vector{Float64}, template_mol::Matrix{Float64}, radii::Vector{Float64}, rs::Float64, prefactors::AbstractVector, overlap_jump::Float64, overlap_slope::Float64, delaunay_eps::Float64)
    n_mol = length(x) ÷ 6
    n_atoms_per_mol = size(template_mol)[2]
    flat_realization = get_flat_realization(x, template_mol)
    MorphoMol.Energies.solvation_free_energy(flat_realization, n_atoms_per_mol, radii, rs, prefactors, overlap_jump, overlap_slope, delaunay_eps)
end

function solvation_free_energy_and_measures(x::Vector{Float64}, template_mol::Matrix{Float64}, radii::Vector{Float64}, rs::Float64, prefactors::AbstractVector, overlap_jump::Float64, overlap_slope::Float64, delaunay_eps::Float64)
    n_mol = length(x) ÷ 6
    n_atoms_per_mol = size(template_mol)[2]
    flat_realization = get_flat_realization(x, template_mol)
    measures = MorphoMol.Energies.get_geometric_measures_and_overlap_value(flat_realization, n_atoms_per_mol, radii, rs, overlap_jump, overlap_slope, delaunay_eps)
    sum(measures .* [prefactors; 1.0]), measures
end

solvation_free_energy_and_measures (generic function with 1 method)

In [5]:
@load "assets/input/jld2/2_6sag_protor_hardsphere_assembly.jld2" template_mol template_radii x_init
n_atoms_per_mol = length(template_mol) ÷ 3
n_mol = length(x_init) ÷ 6
template_mol = reshape(template_mol,(3,n_atoms_per_mol))
radii = vcat([template_radii for i in 1:n_mol]...);

In [6]:
MorphoMol.Utilities.state_to_poly(get_flat_realization(x_init, template_mol), radii, "assets/output/start", n_mol, n_atoms_per_mol)

In [8]:
T = 2.5
ε = 0.01
L = 5
β = 1.0 / T

rs = 1.4
eta = 0.3665
pf = MorphoMol.Energies.get_prefactors(rs, eta)
overlap_slope = 0.85

σ_r = 0.5
σ_t = 0.5

Σ = vcat([[σ_r, σ_r, σ_r, σ_t, σ_t, σ_t] for _ in 1:n_mol]...)

energy(x) = solvation_free_energy_and_measures(x, template_mol, radii, rs, pf, 0.0, overlap_slope, 1.0)
energy_gradient!(∇E, x) = solvation_free_energy_gradient!(∇E, x, template_mol, radii, rs, pf, overlap_slope)

output = MorphoMol.Algorithms.MorphometricSimulationOutput(
    Vector{Vector{Float64}}([]),
    Vector{Float64}([]),
    Vector{Float32}([]),
    Vector{Float32}([]),
    Vector{Float32}([]),
    Vector{Float32}([]),
    Vector{Float32}([]),
    Vector{Float32}([])
)

hmc = MorphoMol.Algorithms.HamiltonianMonteCarlo(energy, energy_gradient!, MorphoMol.Algorithms.standard_leapfrog!, β, L, ε, Σ)
iterations = 10
MorphoMol.Algorithms.simulate!(hmc, output, deepcopy(x_init), iterations);
output.αs[end]

0.2f0

In [9]:
println("Initial energy: $(output.Es[1]) | final energy: $(output.Es[end]) | minimal energy: $(minimum(output.Es))")

Initial energy: 8148.573229333724 | final energy: 8148.8210398944275 | minimal energy: 8148.573229333724


In [17]:
rm("assets/output", recursive = true)
mkpath("assets/output")
for (i, state) in enumerate(output.states)
    MorphoMol.Utilities.state_to_poly(get_flat_realization(state, template_mol), radii, "assets/output/$(i)", n_mol, n_atoms_per_mol)
end

In [10]:
function save_output_plots(sim_out, save_dir)
    paired_colors = cgrad(:Paired_12, 12, categorical = true)
    xs = 1:length(sim_out.Vs)

    pt_per_in = 72
    width =  5*5
    height = 5*3
    f_size = (pt_per_in * width, pt_per_in * height)
    fs = 15
    border_width = 0.5
    fig = Figure(size = f_size, fontsize = fs)
    ms = 4
    scatter(fig[1, 1], xs, sim_out.Vs, markersize = ms, color = paired_colors[1], axis = (ylabel = L"V", ylabelrotation = 0,))
    scatter(fig[1, 2], xs, sim_out.As, markersize = ms, color = paired_colors[3], axis = (ylabel = L"A", ylabelrotation = 0,))
    scatter(fig[1, 3], xs, sim_out.Ms, markersize = ms, color = paired_colors[5], axis = (ylabel = L"M", ylabelrotation = 0,))
    scatter(fig[1, 4], xs, sim_out.Xs, markersize = ms, color = paired_colors[7], axis = (ylabel = L"X", ylabelrotation = 0,))
    scatter(fig[2, 1], xs, sim_out.OLs, markersize = ms, color = paired_colors[10], axis = (ylabel = L"OL", ylabelrotation = 0,))
    scatter(fig[2, 2], xs, sim_out.Es, markersize = ms, color = paired_colors[12], axis = (ylabel = L"F_{sol}", ylabelrotation = 0,))
    save(string("$(save_dir)/output_plots.pdf"), fig, pt_per_unit = 1)
end

save_output_plots (generic function with 1 method)

In [11]:
save_output_plots(output, "notebooks/plots")

CairoMakie.Screen{PDF}
